In [1]:
%cd /Users/santiago/thesis/code

# reload magic
%load_ext autoreload
%autoreload 2

from src import utils
from src.dtw import dtw
from src import jit_dtw
# import dtw as g_dtw

DATASET = 'Beef' # seq length 470
# DATASET = 'ItalyPowerDemand' # seq length 24
# DATASET = 'LargeKitchenAppliances' # seq length 720

S = utils.get_class_sequences(2, DATASET)
a = S[0,:70]
b = S[1,:70]

/Users/santiago/thesis/code
Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [21]:
%time _ = jit_dtw.dtw(a, b)
# utils.plot_alignment(wp, a, b)

CPU times: user 1.04 ms, sys: 7 µs, total: 1.05 ms
Wall time: 1.07 ms
CPU times: user 873 µs, sys: 81 µs, total: 954 µs
Wall time: 1.01 ms
